In [19]:
import pandas as pd

# Load the data
tsa_data = pd.read_csv("/Users/matanlevy/Desktop/Group-1-MMC/Data/csvdata/tsa_final.csv")
weather_data = pd.read_csv("/Users/matanlevy/Desktop/Group-1-MMC/Data/csvdata/weather_data_final.csv")

# Drop duplicates from weather_data based on AirportCode to ensure that each airport code has unique lat/long
weather_data = weather_data.drop_duplicates(subset=['AirportCode'])

# Merge tsa_data and weather_data on 'airportcode'/'AirportCode' column
merged_data = pd.merge(tsa_data, weather_data[['AirportCode','Latitude', 'Longitude']], 
                       left_on='airportcode', 
                       right_on='AirportCode', 
                       how='left')

# Drop 'AirportCode' column from merged_data
merged_data = merged_data.drop(['AirportCode'], axis=1)

# Drop rows with NaN in either 'Latitude' or 'Longitude'
merged_data.dropna(subset=['Latitude', 'Longitude'], inplace=True)

In [6]:
print(merged_data.columns)

Index(['num', 'date', 'hour', 'airportcode', 'airportname', 'city', 'state',
       'checkpoint', 'total', 'Latitude', 'Longitude'],
      dtype='object')


In [13]:
import pandas as pd
import snowflake.connector

# Establish connection to Snowflake
try:
    con = snowflake.connector.connect(
        user='MATANLEVY',
        password='xxxxxxxx',
        account='xxxxxxxx',
        warehouse='COMPUTE_WH',
        database='TSA_MMC',
        schema='INSTANCE'
    )
    # Create a cursor object
    with con.cursor() as cur:
        # Iterate over DataFrame rows
        for i, row in merged_data.iterrows():
            query = """
                INSERT INTO TSA_MMC.INSTANCE.AIRPORTDIMENSION 
                ("AIRPORT ID", "AIRPORT NAME", CITY, STATE, LATITUDE, LONGITUDE, CHECKPOINT) 
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            """
            data = (row['airportcode'], row['airportname'], row['city'], row['state'], row['Latitude'], row['Longitude'], row['checkpoint'])

            # Insert row into Snowflake table
            cur.execute(query, data)
except Exception as e:
    print("Error: ", e)
finally:
    con.close()


KeyboardInterrupt: 

In [25]:
import pandas as pd

# Load the tsa_final.csv file
df_tsa = pd.read_csv('/Users/matanlevy/Desktop/Group-1-MMC/Data/csvdata/tsa_final.csv')

# Convert 'date' column to datetime format
df_tsa['date'] = pd.to_datetime(df_tsa['date'])

# Extract 'hour' from 'hour' column
df_tsa['hour'] = pd.to_datetime(df_tsa['hour'], format='%H:%M').dt.hour

# Combine 'date' and 'hour' to form 'hourdateid'
df_tsa['hourdateid'] = df_tsa['date'].astype(str) + 'T' + df_tsa['hour'].astype(str).str.zfill(2) + ':00'

df_tsa['dayofweek'] = df_tsa['date'].dt.day_name()
df_tsa['month'] = df_tsa['date'].dt.month_name()
df_tsa['year'] = df_tsa['date'].dt.year

# Save the updated dataframe back to csv
print(df_tsa.columns)
print(df_tsa.head)

Index(['num', 'date', 'hour', 'airportcode', 'airportname', 'city', 'state',
       'checkpoint', 'total', 'hourdateid', 'dayofweek', 'month', 'year'],
      dtype='object')
<bound method NDFrame.head of         num       date  hour airportcode                          airportname  \
0         1 2023-05-28     0         ANC  Ted Stevens Anchorage International   
1         2 2023-05-28     0         ATL     Hartsfield Atlanta International   
2         3 2023-05-28     0         BQN                     Rafael Hernandez   
3         4 2023-05-28     0         BTV             Burlington International   
4         5 2023-05-28     0         DCA           Washington Reagan National   
...     ...        ...   ...         ...                                  ...   
213563   27 2023-06-24    23         SLC         Salt Lake City International   
213564   28 2023-06-24    23         SMF     Sacramento International Airport   
213565   29 2023-06-24    23         SMF     Sacramento Internation

In [39]:
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
import pandas as pd
import snowflake.connector

con = snowflake.connector.connect(
    user='MATANLEVY',
    password='xxxxxxx',
    account='xxxxxxx',
    warehouse='COMPUTE_WH',
    database='TSA_MMC',
    schema='INSTANCE'
)

# Configure the connection
engine = create_engine(URL(
    account = 'xxxxxxx',
    user = 'MATANLEVY',
    password = 'xxxxxxxx',
    database = 'TSA_MMC',
    schema = 'INSTANCE',
    warehouse = 'COMPUTE_WH',
    role='ACCOUNTADMIN',
))

# Prepare the data to match the snowflake schema
df_snowflake = df_tsa[['hourdateid', 'date', 'hour', 'dayofweek', 'month', 'year']].copy()
df_snowflake.columns = ["HOUR DATE ID", "DATE", "HOUR OF DAY", "DAY OF WEEK", "MONTH", "YEAR"]

# Make sure types match with snowflake schema
df_snowflake["DATE"] = pd.to_datetime(df_snowflake["DATE"])

# Convert 'hour' to integer type
df_snowflake['HOUR OF DAY'] = df_snowflake['HOUR OF DAY'].astype(int)

df_snowflake["YEAR"] = df_snowflake["YEAR"].astype(int)

chunksize = 10000

# Deduplicate the data
df_snowflake = df_snowflake.drop_duplicates()

# Write the DataFrame to Snowflake
df_snowflake.to_sql('DATEDIMENSIONS', con = engine, if_exists = 'append', index = False, chunksize=chunksize)

/var/folders/ny/mvlwnjvx33lb6dljv2sm_brm0000gn/T/ipykernel_2925/154450391.py:44: UserWarning: The provided table name 'DATEDIMENSIONS' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df_snowflake.to_sql('DATEDIMENSIONS', con = engine, if_exists = 'append', index = False, chunksize=chunksize)


672

In [40]:
import pandas as pd

# Load dataframes
df_weather = pd.read_csv('/Users/matanlevy/Desktop/Group-1-MMC/Data/csvdata/weather_data_final.csv')
df_tsa = pd.read_csv('/Users/matanlevy/Desktop/Group-1-MMC/Data/csvdata/tsa_final.csv')

# Convert 'date' and 'hour' to datetime and format it to match 'time' in df_weather
df_tsa['time'] = pd.to_datetime(df_tsa['date'] + ' ' + df_tsa['hour']).dt.strftime('%Y-%m-%dT%H:%M')

# Merge dataframes on 'time' and 'airportcode'/'AirportCode'
df_merged = pd.merge(df_weather, df_tsa, left_on=['time', 'AirportCode'], right_on=['time', 'airportcode'], how='inner')

print(df_merged.columns)

Index(['time', 'temperature_2m', 'relativehumidity_2m', 'dewpoint_2m',
       'apparent_temperature', 'precipitation_probability', 'precipitation',
       'rain', 'showers', 'snowfall', 'snow_depth', 'cloudcover',
       'AirportCode', 'Latitude', 'Longitude', 'num', 'date', 'hour',
       'airportcode', 'airportname', 'city', 'state', 'checkpoint', 'total'],
      dtype='object')


In [1]:
import random

# Create final dataframe for snowflake table
df_final = pd.DataFrame()
df_final['FACT ID'] = [random.randint(10000,99999) for _ in range(len(df_merged))] # 5 digit random number
df_final['TEMPERATURE'] = df_merged['temperature_2m']
df_final['HUMIDITY'] = df_merged['relativehumidity_2m']
df_final['APPARENT TEMPERATURE'] = df_merged['apparent_temperature']
df_final['PRECIPITATION RAIN'] = df_merged['precipitation']
df_final['RAIN'] = df_merged['rain']
df_final['SNOWFALL'] = df_merged['snowfall']
df_final['SNOW DEPTH'] = df_merged['snow_depth']
df_final['CLOUD COVER TOTAL'] = df_merged['cloudcover']
df_final['HOUR DATE ID'] = pd.to_datetime(df_merged['time']).dt.strftime('%Y%m%d%H%M') # Create ID from time
df_final['AIRPORT ID'] = df_merged['AirportCode'] # Using AirportCode as AIRPORT ID
df_final['TOTAL PAX'] = df_merged['total']

# Push final dataframe to Snowflake
df_final.to_sql('FACTTABLE', con=engine, if_exists='append', index=False, chunksize=chunksize)

NameError: name 'pd' is not defined